In [1]:
%matplotlib qt
import numpy as np
import scipy.constants as sc
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import motSim

$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$
$\newcommand{\bra}[1]{\left\langle{#1}\right|}$

In [2]:
muB=9.274e-24
AMU=1.66e-27
m171 = 171*AMU
Is556 = (0.139e-3)*(1e4)
Is399 = (59.97e-3)*(1e4)
k556 = 2*np.pi/(556e-9)
k399 = 2*np.pi/(399e-9)
gamma556 = 2*np.pi*180e3
gamma399 = 2*np.pi*30e6
Td = sc.hbar*gamma556/(2*sc.k)
vd = np.sqrt(sc.hbar*gamma556/(2*m171))

capture velocity estimate:

$$
\frac{1}{2} m v^2 = \frac{\hbar k \Gamma}{2} d
$$

$$
v_c = \sqrt{\frac{\hbar k \Gamma d}{m}}
$$

$$
T_d = \frac{\hbar \Gamma}{2 k_b}
$$

$$
v_d = \sqrt{\frac{k_b T_d}{m}} = \sqrt{ \frac{\hbar \Gamma}{2 m} }
$$

In [6]:
def vcMOT(d):
    return np.sqrt(sc.hbar*k556*gamma556*d/m171)

## quadrapole field

$$
B_x = G_x x \\
B_y= G_y y \\
B_z = G_z z
$$

$$
G_z = -2 G_x = -2 G_y
$$

## useful paper for MOT forces:

https://arxiv.org/pdf/0905.2897.pdf

## MOT force expressions

When the atom is displaced from the z axis, the quantization axis won't be along axis of the beam and the polarization of each beam will be a mixture of $\sigma^-$, $\sigma^+$, and $\pi$. The polarization (in the $\sigma^-$, $\pi$, $\sigma^+$-basis) can be rotated around y and z using the spin-1 rotation matrices,

$$
U_y(\theta) = \left( \begin{array} ~\frac{1}{2} (1+cos(\theta)) & -\frac{1}{\sqrt{2}}sin(\theta) & \frac{1}{2}(1-cos(\theta)) \\
\frac{1}{\sqrt{2}}sin(\theta) & cos(\theta) & -\frac{1}{\sqrt{2}}sin(\theta) \\
\frac{1}{2} (1-cos(\theta)) & \frac{1}{\sqrt{2}}sin(\theta) & \frac{1}{2}(1+cos(\theta))
\end{array} \right)
$$

$$
U_z(\phi) = \left( \begin{array} ~e^{-i \phi} & 0 & 0 \\
0 & 1 & 0) \\
0 & 0 & e^{i \phi}
\end{array} \right)
$$

In a frame rotated relative to a right-handed beam by $\theta$, (-$\theta$?, doesn't matter I think)
$$
\frac{1}{2}(1+cos(\theta)) \ket{\sigma^+} + \frac{1}{\sqrt{2}}sin(\theta)\ket{\pi} + \frac{1}{2}(1-cos(\theta))\ket{\sigma^-}
$$

break each beam into its polarization components relative to $\boldsymbol{\mu'}$. Right-handed beam (relative to propagation direction of $\mathbf{k_i}$) gives a force:

$$
\mathbf{F_{ki}}(\mathbf{r},\mathbf{k_i}) = \frac{\hbar \mathbf{k_i} \Gamma s_i(\mathbf{r})}{2} \left[ \frac{\left(\frac{1}{2}(1-cos(\theta))\right)^2}{1+s_{tot}(\mathbf{r})+\left( \frac{2 (\delta - \mathbf{k_i}\cdot \mathbf{v} + \mu'B(\mathbf{r})/\hbar)}{\Gamma} \right)^2} + \frac{\frac{1}{2}(1-cos^2(\theta))}{1+s_{tot}(\mathbf{r})+\left( \frac{2 (\delta - \mathbf{k_i}\cdot \mathbf{v})}{\Gamma} \right)^2} 
+ \frac{\left(\frac{1}{2}(1+cos(\theta))\right)^2}{1+s_{tot}(\mathbf{r})+\left( \frac{2 (\delta - \mathbf{k_i}\cdot \mathbf{v} - \mu'B(\mathbf{r})/\hbar)}{\Gamma} \right)^2} \right]
$$

where $\theta$ is the angle between $\mathbf{k_1}$ and $\boldsymbol{\mu'}$, $s_i$ is the beam intensity divided by the saturation intensity of the transition $s_i = I_i/I_s$ at position $\mathbf{r}$, $s_{tot}$ is the sum of all $s_i$ at $\mathbf{r}$, $\delta$ is the detuning, and $\Gamma$ the transition width. Sign in front of Zeeman detuning is pretty confusing— left term is shift of $\Delta m_F = -1$, middle term is shift of $\Delta m_F = 0$, right term is shift of $\Delta m_F = +1$ transition. When $\theta=\pi$, $\mathbf{B}$ and $\boldsymbol{\mu'}$ point opposite $\mathbf{k_1}$ and the right-handed beam (relative to $\mathbf{k_1}$) looks left-handed to the atom, driving $\Delta m_F = -1$ giving the shift $+\mu'B/\hbar$.

## zeeman slower eqns

constant acceleration field profile:
$$
B(z) = B_0 + B_a \sqrt{1-\frac{z}{z_0}}
$$

where $B_a = \frac{\hbar k v_i}{\mu'}$, $z_0 = \frac{v_i^2}{2a}$, $a=\frac{s_0}{1+s_0} a_{max}$, and $B_0 = \frac{\hbar\delta}{\mu'}$

In [9]:
amax= sc.hbar*k399*gamma399/(2*m171)
a = 0.1*amax
vi = 25
z0 = (vi**2)/(2*a)
Ba = sc.hbar*k399*25/muEff
B0 = -sc.hbar*5*gamma399/muEff
print((1e4)*B0, (1e2)*Ba/2, amax, z0)

-71.44781151606642 0.14922266398510112 551378.7893914512 0.0056676101078335225


# pyramid MOT, 1 blue slower

4 green beams in pyramid shape through bottom of cell, 1 blue beam as a Zeeman slower pointing towards diff pumping tube

In [3]:
w0 = 7e-3
delta556 = -30*gamma556
delta399 = -3*gamma399
S556 = 50
S399 = 0.024
Gz = -(3e-2)#z fieldGrad
muEff = 1.5*muB
zeemanTheta = 2.1*np.pi/180
zeemanX0 = -50e-3
zeemanZ0 = 0
zeemanWidth = 4e-3
Natoms = 1
x0 = -220e-3
z0 = 3e-3
vx0 = 25
Tx0 = 0#m171*(7**2)/sc.k
Trho0 = 0#1e-4
NtimeSteps = 600
dt = 100e-6
v0 = np.array([np.random.normal(vx0, np.sqrt(sc.k*Tx0/m171), Natoms),
               np.random.normal(0, np.sqrt(sc.k*Trho0/m171), Natoms),
               np.random.normal(0, np.sqrt(sc.k*Trho0/m171), Natoms)])
r0 = np.array([np.zeros(Natoms)+x0, np.zeros(Natoms), np.zeros(Natoms)+z0])

In [8]:
kb1 = motSim.motBeam(k556*np.array([1/np.sqrt(2), 0, 1/np.sqrt(2)]), w0, np.zeros(3), gamma556, delta556, S556, 1)
kb2 = motSim.motBeam(k556*np.array([-1/np.sqrt(2), 0, 1/np.sqrt(2)]), w0, np.zeros(3), gamma556, delta556, S556, 1)
kb3 = motSim.motBeam(k556*np.array([0, 1/np.sqrt(2), 1/np.sqrt(2)]), w0, np.zeros(3), gamma556, delta556, S556, 1)
kb4 = motSim.motBeam(k556*np.array([0, -1/np.sqrt(2), 1/np.sqrt(2)]), w0, np.zeros(3), gamma556, delta556, S556, 1)
kb5 = motSim.motBeam(k399*np.array([-np.cos(zeemanTheta), 0, np.sin(zeemanTheta)]), zeemanWidth, 
                      np.array([zeemanX0, 0, zeemanZ0]), gamma399, delta399, S399, -1)
kb6 = motSim.motBeam(k399*np.array([-np.cos(zeemanTheta), 0, -np.sin(zeemanTheta)]), zeemanWidth, 
                      np.array([zeemanX0, 0, -zeemanZ0]), gamma399, delta399, S399/3, -1)
kis = np.array([kb1, kb2, kb3, kb4, kb5])#, kb6])

In [5]:
rnAll, vnAll, cFrac = motSim.RK4MC(kis, m171, muEff, Gz, r0, v0, dt, NtimeSteps, Natoms)

capture fraction = 0.000


In [6]:
beamlen = 50
plotSize = 60
balpha = 0.08
fig = plt.figure(figsize=[6,6])
ax = fig.gca(projection='3d')
ax.set_xlim([-plotSize/2,plotSize/2])
ax.set_ylim([-plotSize/2,plotSize/2])
ax.set_zlim([-plotSize/2,plotSize/2])
motSim.plotBeams(ax, kis[:-1], plotSize/2, beamlen, beamWidth=w0*(1e3), colors=np.repeat('#9FCC3B',5), alpha=0.1)
motSim.plotBeams(ax, kis[-1:], plotSize, beamlen*5, beamWidth=zeemanWidth*(1e3), colors=np.repeat('#1BAAD5',5), alpha=0.08)

for i in range(Natoms):
    ax.plot(rnAll[::10,0,i]*(1e3), rnAll[::10,1,i]*(1e3), rnAll[::10,2,i]*(1e3), color='k', alpha=0.1)
    ax.scatter(rnAll[-1,0,i]*(1e3), rnAll[-1,1,i]*(1e3), rnAll[-1,2,i]*(1e3), c='#CC4129', s=2, alpha=0.5)
#     ax.scatter(rnAll[-1,1,i]*(1e3), rnAll[-1,2,i]*(1e3), zs=ax.get_xlim()[0], zdir='x', c='k', s=2, alpha=0.2)
#     ax.scatter(rnAll[-1,0,i]*(1e3), rnAll[-1,2,i]*(1e3), zs=ax.get_ylim()[1], zdir='y', c='k', s=2, alpha=0.2)
#     ax.scatter(rnAll[-1,0,i]*(1e3), rnAll[-1,1,i]*(1e3), zs=ax.get_zlim()[0], zdir='z', c='k', s=2, alpha=0.2)
ax.set_xlabel('x (mm)')
ax.set_ylabel('y (mm)')
ax.set_zlabel('z (mm)')
ax.w_xaxis.set_pane_color((0,0,0,0.01))
ax.w_yaxis.set_pane_color((0,0,0,0.01))
ax.w_zaxis.set_pane_color((0,0,0,0.01))

In [8]:
fig, ax = plt.subplots()
plt.plot(rnAll[:,0,0],vnAll[:,0,0])

In [9]:
fig, ax = plt.subplots()
ax.hist(v0[0]);

In [133]:
z = np.linspace(-20e-3,20e-3,200)
Fkbtest = motSim.FMOT(m171, muEff, Gz, np.array([np.zeros_like(z), np.zeros_like(z), z]),np.zeros((3,200)),kis)
fig, ax = plt.subplots()
plt.plot(z*(1e3), Fkbtest[2])
plt.xlabel('z (mm)')
plt.ylabel('F (N)')
plt.ylim([-5e-23,5e-23])
plt.subplots_adjust(left=0.2, bottom=0.2)

In [15]:
x = np.linspace(-200e-3,20e-3,200)
Fkb556test = motSim.FMOT(m171, muEff, Gz, np.array([x, np.zeros_like(x), np.zeros_like(x)]),np.zeros((3,len(x))),kis[:-2])
Fkb399test = motSim.FMOT(m171, muEff, Gz, np.array([x, np.zeros_like(x), np.zeros_like(x)+(0e-3)]),np.array([np.ones_like(x)*25, np.zeros_like(x), np.zeros_like(x)]),kis[-2:])
Fkb399testv10 = motSim.FMOT(m171, muEff, Gz, np.array([x, np.zeros_like(x), np.zeros_like(x)]),np.array([np.ones_like(x)*10, np.zeros_like(x), np.zeros_like(x)]),kis[-2:])
Fkb399testv0 = motSim.FMOT(m171, muEff, Gz, np.array([x, np.zeros_like(x), np.zeros_like(x)]),np.zeros((3,len(x))),kis[-2:])
Fkbtest = motSim.FMOT(m171, muEff, Gz, np.array([x, np.zeros_like(x), np.zeros_like(x)]),np.zeros((3,len(x))),kis)
fig, ax = plt.subplots()
plt.plot(x, Fkb399test[0], label='399, v=25m/s')
plt.plot(x, Fkb556test[0], label='556, v=0 m/s')
plt.plot(x, Fkb399testv0[0], label='399, v=0 m/s')
plt.plot(x, Fkb399testv10[0], label='399, v=10 m/s')
plt.plot(x, Fkbtest[0], label='sum')
plt.legend()

# 5 green orthogonal 1 blue slower MOT

In [22]:
w0 = 7e-3
S556 = 400
S399 = 0.04
delta556 = -25*gamma556
delta399 = -4.5*gamma399
Gz = -(4e-2)#z fieldGrad
muEff = 1.5*muB
zeemanTheta = 3*np.pi/180
zeemanX0 = 10e-3
zeemanZ0 = -3e-3
zeemanWidth = 4e-3
Natoms = 400
x0 = -220e-3
z0 = 0
vx0 = 25
Tx0 = m171*(10**2)/sc.k
Trho0 = 1e-4
NtimeSteps = 1500
dt = 50e-6
v0 = np.array([np.random.normal(vx0, np.sqrt(sc.k*Tx0/m171), Natoms),
               np.random.normal(0, np.sqrt(sc.k*Trho0/m171), Natoms),
               np.random.normal(0, np.sqrt(sc.k*Trho0/m171), Natoms)])
r0 = np.array([np.zeros(Natoms)+x0, np.zeros(Natoms), np.zeros(Natoms)+z0])

In [18]:
kb1 = motSim.motBeam(k556*np.array([1/np.sqrt(2), 1/np.sqrt(2), 0]), w0, np.zeros(3), gamma556, delta556, S556, 1)
kb2 = motSim.motBeam(k556*np.array([-1/np.sqrt(2), 1/np.sqrt(2), 0]), w0, np.zeros(3), gamma556, delta556, S556, 1)
kb3 = motSim.motBeam(k556*np.array([1/np.sqrt(2), -1/np.sqrt(2), 0]), w0, np.zeros(3), gamma556, delta556, S556, 1)
kb4 = motSim.motBeam(k556*np.array([-1/np.sqrt(2), -1/np.sqrt(2), 0]), w0, np.zeros(3), gamma556, delta556, S556, 1)
kb5 = motSim.motBeam(k556*np.array([0, 0, 1]), w0, np.zeros(3), gamma556, delta556, 100, -1)
kb6 = motSim.motBeam(k399*np.array([-np.cos(zeemanTheta), 0, np.sin(zeemanTheta)]), zeemanWidth, 
                      np.array([zeemanX0, 0, zeemanZ0]), gamma399, delta399, S399, -1)
kis = np.array([kb1, kb2, kb3, kb4, kb5, kb6])

In [20]:
rnAll, vnAll, cFrac = motSim.RK4MC(kis, m171, muEff, Gz, r0, v0, dt, NtimeSteps, Natoms)

capture fraction = 0.015


In [28]:
beamlen = 60
plotSize = 60
balpha = 0.08
fig = plt.figure(figsize=[6,6])
ax = fig.gca(projection='3d')
ax.set_xlim([-20,20])
ax.set_ylim([-20,20])
ax.set_zlim([-20,20])
motSim.plotBeams(ax, kis[:-1], plotSize/2, beamlen, beamWidth=w0*(1e3), colors=np.append(np.repeat('#9FCC3B',5),'#1BAAD5'), alpha=0.08)
motSim.plotBeams(ax, [kis[-1]], plotSize, beamlen*2.5, beamWidth=zeemanWidth*(1e3), colors=['#1BAAD5'], alpha=0.08)
for i in range(Natoms):
    ax.plot(rnAll[::10,0,i]*(1e3), rnAll[::10,1,i]*(1e3), rnAll[::10,2,i]*(1e3), color='k', alpha=0.01)
    ax.scatter(rnAll[-1,0,i]*(1e3), rnAll[-1,1,i]*(1e3), rnAll[-1,2,i]*(1e3), c='#CC4129', s=2, alpha=0.5)
#     ax.scatter(rnAll[-1,1,i]*(1e3), rnAll[-1,2,i]*(1e3), zs=ax.get_xlim()[0], zdir='x', c='#CC4129', s=2, alpha=0.2)
#     ax.scatter(rnAll[-1,0,i]*(1e3), rnAll[-1,2,i]*(1e3), zs=ax.get_ylim()[1], zdir='y', c='#CC4129', s=2, alpha=0.2)
#     ax.scatter(rnAll[-1,0,i]*(1e3), rnAll[-1,1,i]*(1e3), zs=ax.get_zlim()[0], zdir='z', c='#CC4129', s=2, alpha=0.2)
ax.set_xlabel('x (mm)')
ax.set_ylabel('y (mm)')
ax.set_zlabel('z (mm)')
ax.w_xaxis.set_pane_color((0,0,0,0.01))
ax.w_yaxis.set_pane_color((0,0,0,0.01))
ax.w_zaxis.set_pane_color((0,0,0,0.01))

In [23]:
fig, ax = plt.subplots()
ax.hist(v0[0], bins=20)
plt.xlabel('$v_x$ (m/s)')
plt.ylabel('num atoms');
plt.subplots_adjust(left=0.2, bottom=0.2)

In [16]:
t = np.arange(0, dt*NtimeSteps, dt*10)
fig, ax = plt.subplots()
plt.plot(t,vnAll[::10,0,0])

In [11]:
fig, ax = plt.subplots()
plt.plot(rnAll[::10,0,0],vnAll[::10,0,0])

# 5 green orthogonal 2 blue slower MOT

In [32]:
w0 = 7e-3
delta556 = -30*gamma556
delta399 = -3*gamma399
Gz = -(6e-2)#z fieldGrad
muEff = 1.5*muB
Natoms = 1
x0 = -100e-3
z0 = 0
vx0 = 25
Tx0 = 0#m171*(5**2)/sc.k
Trho0 = 0#2e-3
NtimeSteps = 2000
dt = 50e-6
v0 = np.array([np.random.normal(vx0, np.sqrt(sc.k*Tx0/m171), Natoms),
               np.random.normal(0, np.sqrt(sc.k*Trho0/m171), Natoms),
               np.random.normal(0, np.sqrt(sc.k*Trho0/m171), Natoms)])
r0 = np.array([np.zeros(Natoms)+x0, np.zeros(Natoms), np.zeros(Natoms)+z0])

In [33]:
S556 = 400
kb1 = motSim.motBeam(k556*np.array([1/np.sqrt(2), 1/np.sqrt(2), 0]), w0, np.zeros(3), gamma556, delta556, S556, 1)
kb2 = motSim.motBeam(k556*np.array([-1/np.sqrt(2), 1/np.sqrt(2), 0]), w0, np.zeros(3), gamma556, delta556, S556, 1)
kb3 = motSim.motBeam(k556*np.array([1/np.sqrt(2), -1/np.sqrt(2), 0]), w0, np.zeros(3), gamma556, delta556, S556, 1)
kb4 = motSim.motBeam(k556*np.array([-1/np.sqrt(2), -1/np.sqrt(2), 0]), w0, np.zeros(3), gamma556, delta556, S556, 1)
kb5 = motSim.motBeam(k556*np.array([0, 0, 1]), w0, np.zeros(3), gamma556, delta556, 100, -1)
zeemanTheta = 10*np.pi/180
zeemanX0 = -5*w0
kb6 = motSim.motBeam(k399*np.array([-np.cos(zeemanTheta), np.sin(zeemanTheta), 0]), w0, 
                      np.array([zeemanX0, 0, 0]), gamma399, delta399, 0.1, -1)
kb7 = motSim.motBeam(k399*np.array([-np.cos(zeemanTheta), -np.sin(zeemanTheta), 0]), w0, 
                      np.array([zeemanX0, 0, 0]), gamma399, delta399, 0.1, -1)
kis = np.array([kb1, kb2, kb3, kb4, kb5, kb6, kb7])

In [34]:
rnAll, vnAll, cFrac = motSim.RK4MC(kis, m171, muEff, Gz, r0, v0, dt, NtimeSteps, Natoms)

capture fraction = 0.000


In [37]:
beamlen = 50
plotSize = 60
balpha = 0.08
fig = plt.figure(figsize=[6,6])
ax = fig.gca(projection='3d')
ax.set_xlim([-20,20])
ax.set_ylim([-20,20])
ax.set_zlim([-20,20])
motSim.plotBeams(ax, kis[:-2], plotSize/2, beamlen, beamWidth=w0*(1e3), colors=np.repeat('#9FCC3B',5), alpha=0.1)
motSim.plotBeams(ax, kis[-2:], plotSize/2, beamlen, beamWidth=zeemanWidth*(1e3), colors=np.repeat('#1BAAD5',2), alpha=0.1)
for i in range(Natoms):
    ax.plot(rnAll[::10,0,i]*(1e3), rnAll[::10,1,i]*(1e3), rnAll[::10,2,i]*(1e3), color='k', alpha=0.05)
    ax.scatter(rnAll[-1,0,i]*(1e3), rnAll[-1,1,i]*(1e3), rnAll[-1,2,i]*(1e3), c='k', s=2, alpha=0.1)
    ax.scatter(rnAll[-1,1,i]*(1e3), rnAll[-1,2,i]*(1e3), zs=ax.get_xlim()[0], zdir='x', c='#CC4129', s=2, alpha=0.2)
    ax.scatter(rnAll[-1,0,i]*(1e3), rnAll[-1,2,i]*(1e3), zs=ax.get_ylim()[1], zdir='y', c='#CC4129', s=2, alpha=0.2)
    ax.scatter(rnAll[-1,0,i]*(1e3), rnAll[-1,1,i]*(1e3), zs=ax.get_zlim()[0], zdir='z', c='#CC4129', s=2, alpha=0.2)
ax.set_xlabel('x (mm)')
ax.set_ylabel('y (mm)')
ax.set_zlabel('z (mm)')
ax.w_xaxis.set_pane_color((0,0,0,0.01))
ax.w_yaxis.set_pane_color((0,0,0,0.01))
ax.w_zaxis.set_pane_color((0,0,0,0.01))

In [122]:
t = np.arange(0, dt*NtimeSteps, dt*10)
fig, ax = plt.subplots()
plt.plot(t,vnAll[::10,0,0])

# 5 orthogonal, 4 horizontal coreshell

In [42]:
w0 = 5e-3
w0Shell = 5e-3
delta556 = -30*gamma556
delta399 = -1.5*gamma399
S556 = 400
S399 = 0.3
Gz = -(6e-2)#z fieldGrad
muEff = 1.5*muB
Natoms = 100
x0 = -300e-3
z0 = 5e-3
vx0 = 25
Tx0 = m171*(7**2)/sc.k
Trho0 = 1e-3
NtimeSteps = 1000
dt = 50e-6
v0 = np.array([np.random.normal(vx0, np.sqrt(sc.k*Tx0/m171), Natoms),
               np.random.normal(0, np.sqrt(sc.k*Trho0/m171), Natoms),
               np.random.normal(0, np.sqrt(sc.k*Trho0/m171), Natoms)])
r0 = np.array([np.zeros(Natoms)+x0, np.zeros(Natoms), np.zeros(Natoms)+z0])

In [43]:
kb1 = motSim.motBeam(k556*np.array([1/np.sqrt(2), 1/np.sqrt(2), 0]), w0, np.zeros(3), gamma556, delta556, S556, 1)
kb2 = motSim.motBeam(k556*np.array([-1/np.sqrt(2), 1/np.sqrt(2), 0]), w0, np.zeros(3), gamma556, delta556, S556, 1)
kb3 = motSim.motBeam(k556*np.array([1/np.sqrt(2), -1/np.sqrt(2), 0]), w0, np.zeros(3), gamma556, delta556, S556, 1)
kb4 = motSim.motBeam(k556*np.array([-1/np.sqrt(2), -1/np.sqrt(2), 0]), w0, np.zeros(3), gamma556, delta556, S556, 1)
kb5 = motSim.motBeam(k556*np.array([0, 0, 1]), w0, np.zeros(3), gamma556, delta556, 200, -1)
kb1b = motSim.motBeamShell(k399*np.array([1/np.sqrt(2), 1/np.sqrt(2), 0]), w0Shell, np.zeros(3), gamma399, delta399, S399, 1)
kb2b = motSim.motBeamShell(k399*np.array([-1/np.sqrt(2), 1/np.sqrt(2), 0]), w0Shell, np.zeros(3), gamma399, delta399, S399, 1)
kb3b = motSim.motBeamShell(k399*np.array([1/np.sqrt(2), -1/np.sqrt(2), 0]), w0Shell, np.zeros(3), gamma399, delta399, S399, 1)
kb4b = motSim.motBeamShell(k399*np.array([-1/np.sqrt(2), -1/np.sqrt(2), 0]), w0Shell, np.zeros(3), gamma399, delta399, S399, 1)
# zeemanTheta = 3.5*np.pi/180
# zeemanX0 = 10e-3
# zeemanWidth = 4e-3
# kb6 = motBeam.motBeam(k399*np.array([-np.cos(zeemanTheta), 0, np.sin(zeemanTheta)]), zeemanWidth, 
#                       np.array([zeemanX0, 0, -3e-3]), gamma399, delta399, 0.05, -1)
kis = np.array([kb1, kb2, kb3, kb4, kb5, kb1b, kb2b, kb3b, kb4b])

In [44]:
rnAll, vnAll, cFrac = motSim.RK4MC(kis, m171, muEff, Gz, r0, v0, dt, NtimeSteps, Natoms)

capture fraction = 0.440


In [45]:
beamlen = 60
plotSize = 60
balpha = 0.08
fig = plt.figure(figsize=[6,6])
ax = fig.gca(projection='3d')
ax.set_xlim([-20,20])
ax.set_ylim([-20,20])
ax.set_zlim([-20,20])
motSim.plotBeams(ax, kis[:-4], plotSize/2, beamlen, beamWidth=w0*(1e3), colors=np.repeat('#9FCC3B',5), alpha=0.1)
motSim.plotBeams(ax, kis[-4:], plotSize, beamlen/1.5, beamWidth=w0Shell*(1e3), colors=np.repeat('#1BAAD5',4), alpha=0.08)
for i in range(Natoms):
    ax.plot(rnAll[::10,0,i]*(1e3), rnAll[::10,1,i]*(1e3), rnAll[::10,2,i]*(1e3), color='k', alpha=0.01)
    ax.scatter(rnAll[-1,0,i]*(1e3), rnAll[-1,1,i]*(1e3), rnAll[-1,2,i]*(1e3), c='#CC4129', s=2, alpha=0.5)
#     ax.scatter(rnAll[-1,1,i]*(1e3), rnAll[-1,2,i]*(1e3), zs=ax.get_xlim()[0], zdir='x', c='#CC4129', s=2, alpha=0.2)
#     ax.scatter(rnAll[-1,0,i]*(1e3), rnAll[-1,2,i]*(1e3), zs=ax.get_ylim()[1], zdir='y', c='#CC4129', s=2, alpha=0.2)
#     ax.scatter(rnAll[-1,0,i]*(1e3), rnAll[-1,1,i]*(1e3), zs=ax.get_zlim()[0], zdir='z', c='#CC4129', s=2, alpha=0.2)
ax.set_xlabel('x (mm)')
ax.set_ylabel('y (mm)')
ax.set_zlabel('z (mm)')
ax.w_xaxis.set_pane_color((0,0,0,0.01))
ax.w_yaxis.set_pane_color((0,0,0,0.01))
ax.w_zaxis.set_pane_color((0,0,0,0.01))

In [33]:
x = np.linspace(-20e-3,20e-3,200)
Fkb556test = motSim.FMOT(m171, muEff, Gz, np.array([x, np.zeros_like(x), np.zeros_like(x)]),np.zeros((3,len(x))),kis[:-2])
Fkb399test = motSim.FMOT(m171, muEff, Gz, np.array([x, np.zeros_like(x), np.zeros_like(x)+(0e-3)]),np.array([np.ones_like(x)*25, np.zeros_like(x), np.zeros_like(x)]),kis[-2:])
Fkb399testv10 = motSim.FMOT(m171, muEff, Gz, np.array([x, np.zeros_like(x), np.zeros_like(x)]),np.array([np.ones_like(x)*10, np.zeros_like(x), np.zeros_like(x)]),kis[-2:])
Fkb399testv0 = motSim.FMOT(m171, muEff, Gz, np.array([x, np.zeros_like(x), np.zeros_like(x)]),np.zeros((3,len(x))),kis[-2:])
Fkbtest = motSim.FMOT(m171, muEff, Gz, np.array([x, np.zeros_like(x), np.zeros_like(x)]),np.zeros((3,len(x))),kis)
fig, ax = plt.subplots()
plt.plot(x, Fkb399test[0], label='399, v=25m/s')
plt.plot(x, Fkb556test[0], label='556, v=0 m/s')
plt.plot(x, Fkb399testv0[0], label='399, v=0 m/s')
plt.plot(x, Fkb399testv10[0], label='399, v=10 m/s')
plt.plot(x, Fkbtest[0], label='sum')
plt.legend()

In [6]:
s = np.linspace(-5*w0, 5*w0, 200)
r = np.array([[-s/np.sqrt(2)], s/np.sqrt(2), np.zeros_like(s)])
fig, ax = plt.subplots()
plt.plot(s*(1e3),kb1b.Sat(r)[0]/S399) #/kb1b.Sat(r)[0,int(len(s)/2)]
plt.plot(s*(1e3),kb1.Sat(r)[0]/S556)
plt.xlabel(r'$\rho$ (mm)')
plt.ylabel('normalized intensity')

Text(0, 0.5, 'normalized intensity')

In [233]:
t = np.arange(0, dt*NtimeSteps, dt*10)
fig, ax = plt.subplots()
plt.plot(t,vnAll[::10,0,0])

In [24]:
fig, ax = plt.subplots()
plt.plot(rnAll[::10,0,0],vnAll[::10,0,0])

In [6]:
x = np.linspace(-50e-3,50e-3,200)
z = x
FkbtestX = motSim.FMOT(m171, muEff, Gz, np.array([x, np.zeros_like(x), np.zeros_like(x)]),np.zeros((3,len(x))),kis)
FkbtestZ = motSim.FMOT(m171, muEff, Gz, np.array([np.zeros_like(z), np.zeros_like(z), z]),np.zeros((3,len(z))),kis)
fig, ax = plt.subplots()
plt.plot(x, FkbtestX[0])
plt.plot(z, FkbtestZ[2])

In [20]:
Natoms = 1
stot = np.zeros(Natoms)
r = np.zeros((3,Natoms))
for ki in kis:
    stot = stot + ki.Sat(r)
stot

array([1700.])

In [25]:
kis[4].Sat(r)

array([100.])

# pyramid MOT w/ 1 green mot beam through objective, 1 blue slower

In [93]:
w0 = 7e-3
S556 = 150
S399 = 0.1
delta556 = -25*gamma556
delta399 = -6*gamma399
Gz = (4e-2)#z fieldGrad
muEff = 1.5*muB
beamTheta = 55*np.pi/180
zeemanTheta = 3*np.pi/180
zeemanX0 = 30e-3
zeemanZ0 = -3e-3
zeemanWidth = 4e-3
Natoms = 1
x0 = -15e-3
z0 = 0
vx0 = 0.5#25
Tx0 = 0#m171*(10**2)/sc.k
Trho0 = 0#1e-3
NtimeSteps = 1500
dt = 50e-6
v0 = np.array([np.random.normal(vx0, np.sqrt(sc.k*Tx0/m171), Natoms),
               np.random.normal(0, np.sqrt(sc.k*Trho0/m171), Natoms),
               np.random.normal(0, np.sqrt(sc.k*Trho0/m171), Natoms)])
r0 = np.array([np.zeros(Natoms)+x0, np.zeros(Natoms), np.zeros(Natoms)+z0])

In [94]:
kb1 = motSim.motBeam(k556*np.array([np.sin(beamTheta), 0, np.cos(beamTheta)]), w0, np.zeros(3), gamma556, delta556, S556, -1)
kb2 = motSim.motBeam(k556*np.array([-np.sin(beamTheta), 0, np.cos(beamTheta)]), w0, np.zeros(3), gamma556, delta556, S556, -1)
kb3 = motSim.motBeam(k556*np.array([0, np.sin(beamTheta), np.cos(beamTheta)]), w0, np.zeros(3), gamma556, delta556, S556, -1)
kb4 = motSim.motBeam(k556*np.array([0, -np.sin(beamTheta), np.cos(beamTheta)]), w0, np.zeros(3), gamma556, delta556, S556, -1)
kb5 = motSim.motBeam(k556*np.array([0, 0, -1]), w0, np.zeros(3), gamma556, delta556, S556*2, 1)
# kb6 = motSim.motBeam(k399*np.array([-np.cos(zeemanTheta), 0, np.sin(zeemanTheta)]), zeemanWidth, 
#                       np.array([zeemanX0, 0, zeemanZ0]), gamma399, delta399, S399, -1)
kis = np.array([kb1, kb2, kb3, kb4, kb5])

In [96]:
rnAll, vnAll, cFrac = motSim.RK4MC(kis, m171, muEff, Gz, r0, v0, dt, NtimeSteps, Natoms)

capture fraction = 1.000


In [97]:
beamlen = 65
plotSize = 60
balpha = 0.08
fig = plt.figure(figsize=[6,6])
ax = fig.gca(projection='3d')
ax.set_xlim([-plotSize/2,plotSize/2])
ax.set_ylim([-plotSize/2,plotSize/2])
ax.set_zlim([-plotSize/2,plotSize/2])
motSim.plotBeams(ax, kis, plotSize/2, beamlen, beamWidth=w0*(1e3), colors=np.repeat('#9FCC3B',5), alpha=0.08)
# motSim.plotBeams(ax, kis[-1:], plotSize, beamlen*5, beamWidth=zeemanWidth*(1e3), colors=np.repeat('#1BAAD5',5), alpha=0.08)

for i in range(Natoms):
    ax.plot(rnAll[::10,0,i]*(1e3), rnAll[::10,1,i]*(1e3), rnAll[::10,2,i]*(1e3), color='k', alpha=0.1)
    ax.scatter(rnAll[-1,0,i]*(1e3), rnAll[-1,1,i]*(1e3), rnAll[-1,2,i]*(1e3), c='#CC4129', s=2, alpha=0.5)
#     ax.scatter(rnAll[-1,1,i]*(1e3), rnAll[-1,2,i]*(1e3), zs=ax.get_xlim()[0], zdir='x', c='k', s=2, alpha=0.2)
#     ax.scatter(rnAll[-1,0,i]*(1e3), rnAll[-1,2,i]*(1e3), zs=ax.get_ylim()[1], zdir='y', c='k', s=2, alpha=0.2)
#     ax.scatter(rnAll[-1,0,i]*(1e3), rnAll[-1,1,i]*(1e3), zs=ax.get_zlim()[0], zdir='z', c='k', s=2, alpha=0.2)
ax.set_xlabel('x (mm)')
ax.set_ylabel('y (mm)')
ax.set_zlabel('z (mm)')
ax.w_xaxis.set_pane_color((0,0,0,0.01))
ax.w_yaxis.set_pane_color((0,0,0,0.01))
ax.w_zaxis.set_pane_color((0,0,0,0.01))

In [69]:
fig, ax = plt.subplots()
plt.plot(rnAll[:,0,0],vnAll[:,0,0])

In [707]:
fig, ax = plt.subplots()
ax.hist(v0[0]);

In [95]:
x = np.linspace(-50e-3,50e-3,200)
z = x
FkbtestX = motSim.FMOT(m171, muEff, Gz, np.array([x, np.zeros_like(x), np.zeros_like(x)]),np.zeros((3,len(x))),kis)
FkbtestZ = motSim.FMOT(m171, muEff, Gz, np.array([np.zeros_like(z), np.zeros_like(z), z]),np.zeros((3,len(z))),kis)
fig, ax = plt.subplots()
plt.plot(x, FkbtestX[0])
plt.plot(z, FkbtestZ[2])
plt.ylim([-1e-22, 1e-22])

(-1e-22, 1e-22)

# 1/2 pyramid MOT w/ 1 green mot beam through objective, green slower

In [833]:
((4+1.5)*100+300)*0.139

118.15

In [836]:
300*0.139

41.7

In [80]:
w0 = 4e-3
S556 = 100
S556sl = 300
delta556 = -8*gamma556
delta556sl = -40*gamma556
Gz = (1.9e-2)#z fieldGrad
muEff = 1.5*muB
beamTheta = 35*np.pi/180
zeemanTheta = 2*np.pi/180
zeemanX0 = 0#30e-3
zeemanZ0 = -3e-3
zeemanWidth = 5e-3
Natoms = 500
x0 = -220e-3
z0 = 0
vx0 = 25
Tx0 = m171*(1**2)/sc.k
Trho0 = 1e-3
NtimeSteps = 1500
dt = 50e-6
v0 = np.array([np.random.normal(vx0, np.sqrt(sc.k*Tx0/m171), Natoms),
               np.random.normal(0, np.sqrt(sc.k*Trho0/m171), Natoms),
               np.random.normal(0, np.sqrt(sc.k*Trho0/m171), Natoms)])
r0 = np.array([np.zeros(Natoms)+x0, np.zeros(Natoms), np.zeros(Natoms)+z0])

In [81]:
kb1 = motSim.motBeam(k556*np.array([np.sin(beamTheta), 0, np.cos(beamTheta)]), w0, np.zeros(3), gamma556, delta556, S556, -1)
kb2 = motSim.motBeam(k556*np.array([-np.sin(beamTheta), 0, np.cos(beamTheta)]), w0, np.zeros(3), gamma556, delta556, S556, -1)
kb3 = motSim.motBeam(k556*np.array([0, 1, 0]), w0, np.zeros(3), gamma556, delta556, S556, -1)
kb4 = motSim.motBeam(k556*np.array([0, -1, 0]), w0, np.zeros(3), gamma556, delta556, S556, -1)
kb5 = motSim.motBeam(k556*np.array([0, 0, -1]), w0, np.zeros(3), gamma556, delta556, S556*1.5, 1)
kb6 = motSim.motBeam(k556*np.array([-np.cos(zeemanTheta), 0, np.sin(zeemanTheta)]), zeemanWidth, 
                      np.array([zeemanX0, 0, zeemanZ0]), gamma556, delta556sl, S556sl, -1)
kis = np.array([kb1, kb2, kb3, kb4, kb5, kb6])

In [56]:
rnAll, vnAll, cFrac = motSim.RK4MC(kis, m171, muEff, Gz, r0, v0, dt, NtimeSteps, Natoms)

capture fraction = 0.078


In [41]:
figF, axF = plt.subplots()

In [79]:
x = np.linspace(-220e-3,100e-3,200)
z = x
# FkbtestXsl = motSim.FMOT(m171, muEff, Gz, np.array([x, np.zeros_like(x), np.zeros_like(x)]),np.zeros((3,len(x))),kis[5])
FkbtestX = motSim.FMOT(m171, muEff, Gz, np.array([x, np.zeros_like(x), np.zeros_like(x)]),np.zeros((3,len(x))),kis)
FkbtestXv = motSim.FMOT(m171, muEff, Gz, np.array([x, np.zeros_like(x), np.zeros_like(x)]),
                        np.array([np.ones_like(x)*10, np.zeros_like(x), np.zeros_like(x)]),kis)
FkbtestXv2 = motSim.FMOT(m171, muEff, Gz, np.array([x, np.zeros_like(x), np.zeros_like(x)]),
                        np.array([np.ones_like(x)*20, np.zeros_like(x), np.zeros_like(x)]),kis)
FkbtestXv3 = motSim.FMOT(m171, muEff, Gz, np.array([x, np.zeros_like(x), np.zeros_like(x)]),
                        np.array([np.ones_like(x)*30, np.zeros_like(x), np.zeros_like(x)]),kis)
# FkbtestZ = motSim.FMOT(m171, muEff, Gz, np.array([np.zeros_like(z), np.zeros_like(z), z]),np.zeros((3,len(z))),kis)
plt.figure(figF.number)
plt.clf()
axF = plt.axes()
plt.plot(x, FkbtestX[0], label='Fx (v=0m/s)')
plt.plot(x, FkbtestXv[0], label='Fx (v=7m/s)')
plt.plot(x, FkbtestXv2[0], label='Fx (v=15m/s)')
plt.plot(x, FkbtestXv3[0], label='Fx (v=25m/s)')
plt.legend()
# plt.plot(z, FkbtestZ[2])
# plt.ylim([-1e-21, 1e-21])

In [230]:
fig = plt.figure(figsize=[6,6])

In [58]:
beamlen = 65
plotSize = 60
balpha = 0.08
plt.figure(fig.number)
plt.clf()
ax = fig.gca(projection='3d')
ax.set_xlim([-plotSize/2,plotSize/2])
ax.set_ylim([-plotSize/2,plotSize/2])
ax.set_zlim([-plotSize/2,plotSize/2])
motSim.plotBeams(ax, kis, plotSize/2, beamlen, beamWidth=w0*(1e3), colors=np.repeat('#9FCC3B',5), alpha=0.08)
# motSim.plotBeams(ax, kis[-1:], plotSize, beamlen*5, beamWidth=zeemanWidth*(1e3), colors=np.repeat('#1BAAD5',5), alpha=0.08)

for i in range(Natoms):
    ax.plot(rnAll[::10,0,i]*(1e3), rnAll[::10,1,i]*(1e3), rnAll[::10,2,i]*(1e3), color='k', alpha=0.01)
    ax.scatter(rnAll[-1,0,i]*(1e3), rnAll[-1,1,i]*(1e3), rnAll[-1,2,i]*(1e3), c='#CC4129', s=2, alpha=0.5)
#     ax.scatter(rnAll[-1,1,i]*(1e3), rnAll[-1,2,i]*(1e3), zs=ax.get_xlim()[0], zdir='x', c='k', s=2, alpha=0.2)
#     ax.scatter(rnAll[-1,0,i]*(1e3), rnAll[-1,2,i]*(1e3), zs=ax.get_ylim()[1], zdir='y', c='k', s=2, alpha=0.2)
#     ax.scatter(rnAll[-1,0,i]*(1e3), rnAll[-1,1,i]*(1e3), zs=ax.get_zlim()[0], zdir='z', c='k', s=2, alpha=0.2)
ax.set_xlabel('x (mm)')
ax.set_ylabel('y (mm)')
ax.set_zlabel('z (mm)')
ax.w_xaxis.set_pane_color((0,0,0,0.01))
ax.w_yaxis.set_pane_color((0,0,0,0.01))
ax.w_zaxis.set_pane_color((0,0,0,0.01))

In [228]:
figR, axR = plt.subplots()

In [59]:
ia = 14
Frv = motSim.FMOT(m171, muEff, Gz, np.transpose(rnAll[:,:,ia]), np.transpose(vnAll[:,:,ia]),kis)
plt.figure(figR.number)
plt.clf()
axR = plt.axes()
plt.plot(rnAll[:,0,ia],vnAll[:,0,ia])
plt.ylabel('v (m/s)')
axR2 = axR.twinx()
axR2.plot(rnAll[:,0,ia],Frv[0],'r')
plt.xlabel('x (m)')
plt.ylabel('F (N)')

Text(0, 0.5, 'F (N)')

In [707]:
fig, ax = plt.subplots()
ax.hist(v0[0]);

# 1/2 pyramid MOT w/ 1 green mot beam through objective, blue slower

In [89]:
4*(399e-9)*(0.15)/(np.pi*0.020)

3.810169337619974e-06

In [90]:
np.pi*((4e-6)**2)/(399e-9)

0.00012597865277553055

In [229]:
10/((4e-4)**2)

62500000.0

In [362]:
w0 = 5e-3
S556 = 200
S399 = 4.5e4
delta556 = -15*gamma556
delta399 = -2*gamma399
Gz = (4e-2)#z fieldGrad
muEff = 1.5*muB
beamTheta = 35*np.pi/180
zeemanTheta = 2*np.pi/180
zeemanX0 = -120e-3
zeemanZ0 = 0.5e-3
zeemanWidth = 4e-6
Natoms = 1
x0 = -220e-3
z0 = 0
vx0 = 25
Tx0 = 0#m171*(10**2)/sc.k
Trho0 = 0#1e-3
NtimeSteps = 1000
dt = 50e-6
v0 = np.array([np.random.normal(vx0, np.sqrt(sc.k*Tx0/m171), Natoms),
               np.random.normal(0, np.sqrt(sc.k*Trho0/m171), Natoms),
               np.random.normal(0, np.sqrt(sc.k*Trho0/m171), Natoms)])
r0 = np.array([np.zeros(Natoms)+x0, np.zeros(Natoms), np.zeros(Natoms)+z0])

In [363]:
kb1 = motSim.motBeam(k556*np.array([np.sin(beamTheta), 0, np.cos(beamTheta)]), w0, np.zeros(3), gamma556, delta556, S556, -1)
kb2 = motSim.motBeam(k556*np.array([-np.sin(beamTheta), 0, np.cos(beamTheta)]), w0, np.zeros(3), gamma556, delta556, S556, -1)
kb3 = motSim.motBeam(k556*np.array([0, 1, 0]), w0, np.zeros(3), gamma556, delta556, S556, -1)
kb4 = motSim.motBeam(k556*np.array([0, -1, 0]), w0, np.zeros(3), gamma556, delta556, S556, -1)
kb5 = motSim.motBeam(k556*np.array([0, 0, -1]), w0, np.zeros(3), gamma556, delta556, S556*2, 1)
kb6 = motSim.motBeam(k556*np.array([-np.cos(zeemanTheta), 0, np.sin(zeemanTheta)]), zeemanWidth, 
                      np.array([zeemanX0, 0, zeemanZ0]), gamma399, delta399, S399, -1)
kis = np.array([kb1, kb2, kb3, kb4, kb5, kb6])

In [365]:
rnAll, vnAll, cFrac = motSim.RK4MC(kis, m171, muEff, Gz, r0, v0, dt, NtimeSteps, Natoms)

capture fraction = 0.000


In [129]:
figF, axF = plt.subplots()

In [366]:
x = np.linspace(-220e-3,100e-3,200)
# FkbtestXsl = motSim.FMOT(m171, muEff, Gz, np.array([x, np.zeros_like(x), np.zeros_like(x)]),np.zeros((3,len(x))),kis[5])
FkbtestX = motSim.FMOT(m171, muEff, Gz, np.array([x, np.zeros_like(x), np.zeros_like(x)]),np.zeros((3,len(x))),kis)
FkbtestXv = motSim.FMOT(m171, muEff, Gz, np.array([x, np.zeros_like(x), np.zeros_like(x)]),
                        np.array([np.ones_like(x)*7, np.zeros_like(x), np.zeros_like(x)]),kis)
FkbtestXv2 = motSim.FMOT(m171, muEff, Gz, np.array([x, np.zeros_like(x), np.zeros_like(x)]),
                        np.array([np.ones_like(x)*15, np.zeros_like(x), np.zeros_like(x)]),kis)
FkbtestXv3 = motSim.FMOT(m171, muEff, Gz, np.array([x, np.zeros_like(x), np.zeros_like(x)]),
                        np.array([np.ones_like(x)*25, np.zeros_like(x), np.zeros_like(x)]),kis)
plt.figure(figF.number)
plt.clf()
axF = plt.axes()
plt.plot(x, FkbtestX[0], label='Fx (v=0m/s)')
plt.plot(x, FkbtestXv[0], label='Fx (v=7m/s)')
plt.plot(x, FkbtestXv2[0], label='Fx (v=15m/s)')
plt.plot(x, FkbtestXv3[0], label='Fx (v=25m/s)')
plt.legend()
# plt.plot(z, FkbtestZ[2])
plt.ylim([-4e-21, 1e-21])

(-4e-21, 1e-21)

In [152]:
fig = plt.figure(figsize=[6,6])

In [367]:
beamlen = 65
plotSize = 60
balpha = 0.08
plt.figure(fig.number)
plt.clf()
ax = fig.gca(projection='3d')
ax.set_xlim([-plotSize/2,plotSize/2])
ax.set_ylim([-plotSize/2,plotSize/2])
ax.set_zlim([-plotSize/2,plotSize/2])
motSim.plotBeams(ax, kis[:-1], plotSize/2, beamlen, beamWidth=w0*(1e3), colors=np.repeat('#9FCC3B',5), alpha=0.08)
motSim.plotBeams(ax, kis[-1:], plotSize, beamlen*5, beamWidth=zeemanWidth*(1e4), colors=np.repeat('#1BAAD5',5), alpha=0.08)

for i in range(Natoms):
    ax.plot(rnAll[::10,0,i]*(1e3), rnAll[::10,1,i]*(1e3), rnAll[::10,2,i]*(1e3), color='k', alpha=0.1)
    ax.scatter(rnAll[-1,0,i]*(1e3), rnAll[-1,1,i]*(1e3), rnAll[-1,2,i]*(1e3), c='#CC4129', s=2, alpha=0.5)
#     ax.scatter(rnAll[-1,1,i]*(1e3), rnAll[-1,2,i]*(1e3), zs=ax.get_xlim()[0], zdir='x', c='k', s=2, alpha=0.2)
#     ax.scatter(rnAll[-1,0,i]*(1e3), rnAll[-1,2,i]*(1e3), zs=ax.get_ylim()[1], zdir='y', c='k', s=2, alpha=0.2)
#     ax.scatter(rnAll[-1,0,i]*(1e3), rnAll[-1,1,i]*(1e3), zs=ax.get_zlim()[0], zdir='z', c='k', s=2, alpha=0.2)
ax.set_xlabel('x (mm)')
ax.set_ylabel('y (mm)')
ax.set_zlabel('z (mm)')
ax.w_xaxis.set_pane_color((0,0,0,0.01))
ax.w_yaxis.set_pane_color((0,0,0,0.01))
ax.w_zaxis.set_pane_color((0,0,0,0.01))

In [150]:
figR, axR = plt.subplots()

In [368]:
Frv = motSim.FMOT(m171, muEff, Gz, np.transpose(rnAll[:,:,0]), np.transpose(vnAll[:,:,0]),kis)
plt.figure(figR.number)
plt.clf()
axR = plt.axes()
plt.plot(rnAll[:,0,0],vnAll[:,0,0])
axR2 = axR.twinx()
axR2.plot(rnAll[:,0,0],Frv[0],'r')

In [707]:
fig, ax = plt.subplots()
ax.hist(v0[0]);

# 4 beam pyramid coreshell MOT

In [35]:
w0 = 5e-3
w0Shell = 5e-3
delta556 = -30*gamma556
delta399 = -3*gamma399
S556 = 100
S399 = 0.1
Gz = (2.5e-2)#z fieldGrad
muEff = 1.5*muB
Natoms = 1
x0 = -220e-3
z0 = 3e-3
vx0 = 25
Tx0 = 0#m171*(10**2)/sc.k
Trho0 = 0#1e-4
NtimeSteps = 600
dt = 50e-6
v0 = np.array([np.random.normal(vx0, np.sqrt(sc.k*Tx0/m171), Natoms),
               np.random.normal(0, np.sqrt(sc.k*Trho0/m171), Natoms),
               np.random.normal(0, np.sqrt(sc.k*Trho0/m171), Natoms)])
r0 = np.array([np.zeros(Natoms)+x0, np.zeros(Natoms), np.zeros(Natoms)+z0])

In [36]:
kb1 = motSim.motBeam(k556*np.array([1/np.sqrt(2), 0, 1/np.sqrt(2)]), w0, np.zeros(3), gamma556, delta556, S556, 1)
kb2 = motSim.motBeam(k556*np.array([-1/np.sqrt(2), 0, 1/np.sqrt(2)]), w0, np.zeros(3), gamma556, delta556, S556, 1)
kb3 = motSim.motBeam(k556*np.array([0, 1/np.sqrt(2), 1/np.sqrt(2)]), w0, np.zeros(3), gamma556, delta556, S556, 1)
kb4 = motSim.motBeam(k556*np.array([0, -1/np.sqrt(2), 1/np.sqrt(2)]), w0, np.zeros(3), gamma556, delta556, S556, 1)
kb1b = motSim.motBeamShell(k399*np.array([-1/np.sqrt(2), 0, 1/np.sqrt(2)]), w0Shell, np.zeros(3), gamma399, delta399, S399, 1, l=8)
kb2b = motSim.motBeamShell(k399*np.array([1/np.sqrt(2), 0, 1/np.sqrt(2)]), w0Shell, np.zeros(3), gamma399, delta399, S399, 1, l=8)
kb3b = motSim.motBeamShell(k399*np.array([0, 1/np.sqrt(2), 1/np.sqrt(2)]), w0Shell, np.zeros(3), gamma399, delta399, S399, 1, l=8)
kb4b = motSim.motBeamShell(k399*np.array([0, -1/np.sqrt(2), 1/np.sqrt(2)]), w0Shell, np.zeros(3), gamma399, delta399, S399, 1, l=8)
kis = np.array([kb1, kb2, kb3, kb4, kb1b, kb2b, kb3b, kb4b])

In [37]:
rnAll, vnAll, cFrac = motSim.RK4MC(kis, m171, muEff, Gz, r0, v0, dt, NtimeSteps, Natoms)

capture fraction = 0.000


In [38]:
beamlen = 65
plotSize = 60
balpha = 0.08
fig = plt.figure(figsize=[6,6])
ax = fig.gca(projection='3d')
ax.set_xlim([-plotSize/2,plotSize/2])
ax.set_ylim([-plotSize/2,plotSize/2])
ax.set_zlim([-plotSize/2,plotSize/2])
motSim.plotBeams(ax, kis[:-4], plotSize/2, beamlen, beamWidth=w0*(1e3), colors=np.repeat('#9FCC3B',5), alpha=0.08)
motSim.plotBeams(ax, kis[-4:], plotSize, beamlen/2, beamWidth=w0Shell*(1e3), colors=np.repeat('#1BAAD5',5), alpha=0.08)

for i in range(Natoms):
    ax.plot(rnAll[::10,0,i]*(1e3), rnAll[::10,1,i]*(1e3), rnAll[::10,2,i]*(1e3), color='k', alpha=0.1)
    ax.scatter(rnAll[-1,0,i]*(1e3), rnAll[-1,1,i]*(1e3), rnAll[-1,2,i]*(1e3), c='#CC4129', s=2, alpha=0.5)
#     ax.scatter(rnAll[-1,1,i]*(1e3), rnAll[-1,2,i]*(1e3), zs=ax.get_xlim()[0], zdir='x', c='k', s=2, alpha=0.2)
#     ax.scatter(rnAll[-1,0,i]*(1e3), rnAll[-1,2,i]*(1e3), zs=ax.get_ylim()[1], zdir='y', c='k', s=2, alpha=0.2)
#     ax.scatter(rnAll[-1,0,i]*(1e3), rnAll[-1,1,i]*(1e3), zs=ax.get_zlim()[0], zdir='z', c='k', s=2, alpha=0.2)
ax.set_xlabel('x (mm)')
ax.set_ylabel('y (mm)')
ax.set_zlabel('z (mm)')
ax.w_xaxis.set_pane_color((0,0,0,0.01))
ax.w_yaxis.set_pane_color((0,0,0,0.01))
ax.w_zaxis.set_pane_color((0,0,0,0.01))

In [39]:
fig, ax = plt.subplots()
plt.plot(rnAll[:,0,0],vnAll[:,0,0])

In [707]:
fig, ax = plt.subplots()
ax.hist(v0[0]);

In [60]:
z = np.linspace(-20e-3,20e-3,200)
Fkbtest = motSim.FMOT(m171, muEff, Gz, np.array([np.zeros_like(z), np.zeros_like(z), z]),np.zeros((3,200)),kis)
fig, ax = plt.subplots()
plt.plot(z, Fkbtest[2])
plt.ylim([-5e-23,5e-23])

(-5e-23, 5e-23)

In [41]:
x = np.linspace(-50e-3,50e-3,200)
Fkb556test = motSim.FMOT(m171, muEff, Gz, np.array([x, np.zeros_like(x), np.zeros_like(x)]),np.zeros((3,len(x))),kis)
Fkb399test = motSim.FMOT(m171, muEff, Gz, np.array([x, np.zeros_like(x), np.zeros_like(x)+(0e-3)]),np.array([np.ones_like(x)*25, np.zeros_like(x), np.zeros_like(x)]),kis)
Fkb399testv10 = motSim.FMOT(m171, muEff, Gz, np.array([x, np.zeros_like(x), np.zeros_like(x)]),np.array([np.ones_like(x)*10, np.zeros_like(x), np.zeros_like(x)]),kis)
Fkb399testv0 = motSim.FMOT(m171, muEff, Gz, np.array([x, np.zeros_like(x), np.zeros_like(x)]),np.zeros((3,len(x))),kis)
Fkbtest = motSim.FMOT(m171, muEff, Gz, np.array([x, np.zeros_like(x), np.zeros_like(x)]),np.zeros((3,len(x))),kis)
fig, ax = plt.subplots()
plt.plot(x, Fkb399test[0], label='399, v=25m/s')
# plt.plot(x, Fkb556test[0], label='556, v=0 m/s')
# plt.plot(x, Fkb399testv0[0], label='399, v=0 m/s')
# plt.plot(x, Fkb399testv10[0], label='399, v=10 m/s')
plt.plot(x, Fkbtest[0], label='sum')
plt.legend()

In [10]:
kbt1 = motSim.motBeamShell(k399*np.array([-1/np.sqrt(2), 0, 1/np.sqrt(2)]), 5e-3, np.zeros(3), gamma399, delta399, S399, 1, l=3)
kbt2 = motSim.motBeamShell(k399*np.array([-1/np.sqrt(2), 0, 1/np.sqrt(2)]), 4e-3, np.zeros(3), gamma399, delta399, S399, 1, l=5)
s = np.linspace(-5*w0, 5*w0, 200)
r = np.array([[-s/np.sqrt(2)], s/np.sqrt(2), np.zeros_like(s)])
fig, ax = plt.subplots()
plt.plot(s*(1e3),kbt1.Sat(r)[0]/S399) #/kb1b.Sat(r)[0,int(len(s)/2)]
plt.plot(s*(1e3),kbt2.Sat(r)[0]/S399)
plt.xlabel(r'$\rho$ (mm)')
plt.ylabel('normalized intensity')

Text(0, 0.5, 'normalized intensity')